# MagNav.jl Demonstration Notebook
This notebook provides a demonstration of some of the key functions provided in MagNav.jl: https://github.com/MIT-AI-Accelerator/MagNav.jl

The first step is to bring in some common dataframes provided by the library relevant to flights performed by Sander Geophysics Ltd. (SGL) in 2020.

Dataframe | Description
-------- | --------------
df_map  | Map files for SGL flights
df_comp | SGL compensation flight lines
df_flight | SGL flight files
df_all | All flight lines
df_nav | All *navigation capable* flight lines
df_event | Pilot-recorded flight events

In [ ]:
cd(@__DIR__)
# uncomment line below to use local MagNav.jl (downloaded folder)
# using Pkg; Pkg.activate("../"); Pkg.instantiate()
using MagNav
using CSV, DataFrames
using Plots: plot, plot!
using Random: seed!
using Statistics: mean, median, std
seed!(33); # for reproducibility
include("dataframes_setup.jl"); # setup DataFrames

## Flight line selection
Select a flight and gather the [`XYZ` data structure](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#Flight-Data), which contains the GPS-based trajectory and inertial navigation system [(`Traj`) and (`INS`) data structures](https://mit-ai-accelerator.github.io/MagNav.jl/stable/structs/#Flight-Paths), flight information, magnetometer readings, and auxilliary sensor data.

Note that `XYZ` holds all the data from the HDF5 file, but we can use Boolean indices as a mask to return only the specific flight lines that we want to analyze.


In [ ]:
flight = :Flt1006 # specify flight, full list of SGL flights is in df_flight
xyz    = get_XYZ(flight,df_flight); # load flight data

Here we choose a magnetic map (see df_map for other examples) and then filter
df_nav to ensure that we can select a flight line that both corresponds with
our selected flight (:Flt1006) and exists on the map.

In [ ]:
map_name   = :Eastern_395 # specify map, full list of maps in df_map
df_options = df_nav[(df_nav.flight   .== flight  ) .& # full list of navigation-capable flight lines is in df_nav
                    (df_nav.map_name .== map_name),:] # flight line options that are valid for the selected flight & map

## Get data indices for selected flight line
To test navigation, we'll use (Boolean) indices corresponding to the first flight line (1006.08) from the above table.

In [ ]:

line = df_options.line[1] # select flight line (row) from df_options
ind  = get_ind(xyz,line,df_nav); # get indices

For the Tolles-Lawson (TL) calibration, we'll select Flight 1006, flight line 1006.04, which occurred at a higher altitude (see [readme](https://github.com/MIT-AI-Accelerator/MagNav.jl/blob/master/readmes/Flt1006_readme.txt)). This is the first calibration box of this flight line. TL_ind holds the indices just for this portion of the calibration flight.

In [ ]:
TL_i   = 6 # select first calibration box of 1006.04
TL_ind = get_ind(xyz;tt_lim=[df_comp.t_start[TL_i],df_comp.t_end[TL_i]]); # get indices

## Plotting
To get an idea of the magnetometer data during the calibration / test flight, we can call some MagNav.jl utilities for plotting.

Note that these are filtered using the `ind` indices for line 1006.08.

In [ ]:
show_plot = true
save_plot = false
use_mags  = [:mag_1_uc,:mag_4_uc,:mag_5_uc] # scalar magnetometers to plot

p1 = plot_mag(xyz;ind,show_plot,save_plot, # plot scalar magnetometers
              use_mags=use_mags,
              detrend_data=true,
              plot_png="scalar_mags.png");

In [ ]:
p2 = plot_mag(xyz;ind,show_plot,save_plot, # plot vector magnetometer (fluxgate), Flux D
              use_mags=[:flux_d],
              detrend_data=true,
              plot_png="vector_mag_d.png");

Clearly, the in-cabin scalar and vector magnetometers are noisy, but the stinger (mag_1) matches the map value closely.

## Create the Tolles-Lawson model
Next, we select scalar and vector magnetometer readings during the calibration flight and generate the coefficients to perform linear TL compensation. We're choosing in-cabin scalar magnetometer 4 and vector (flux) magnetometer D. Mag 4 is located on the floor in the rear of the cabin, and Flux D is nearby on the starboard side. Mag 4 is particularly challenging since it contains several 100s to 1000 nT excursions as compared with the tail stinger.

### Tolles-Lawson calibration

In [ ]:
λ       = 0.025   # ridge parameter
use_vec = :flux_d # selected vector (flux) magnetometer 
flux    = getfield(xyz,use_vec) # load Flux D data
TL_d_4  = create_TL_coef(flux,xyz.mag_4_uc,TL_ind;λ=λ); # create Tolles-Lawson coefficients

### Tolles-Lawson compensation (linear model)

In [ ]:
A = create_TL_A(flux,ind)      # Tolles-Lawson "A" matrix for Flux D
mag_1_sgl = xyz.mag_1_c[ind]   # professionally compensated tail stinger, Mag 1
mag_4_uc  = xyz.mag_4_uc[ind]; # uncompensated Mag 4
mag_4_c   = mag_4_uc - detrend(A*TL_d_4;mean_only=true); # compensated Mag 4

## Navigation
Here, the trajectory (`Traj`) and `INS` structures are pulled from the `XYZ` structure for convenience, and map data is loaded into a map (`MapS`) structure. The map is then "upward continued" (via a Fast Fourier Transform) to the trajectory altitude and outfitted with an interpolation function (`itp_mapS`). Finally, the expected scalar magnetometer reading along the flight path (`map_val`) is computed using the interpolation function, as done in `get_map_val()`, then corrected for diurnal effects and the core magnetic field (IGRF).

In [ ]:
traj = get_traj(xyz,ind) # trajectory (GPS) struct
ins  = get_ins( xyz,ind;N_zero_ll=1) # INS struct, "zero" lat/lon to match `traj` for 1 data point
mapS = get_map(map_name,df_map) # load map data
# get map values & map interpolation function
(map_val,itp_mapS) = get_map_val(mapS,traj;return_itp=true)
map_val += (xyz.diurnal + xyz.igrf)[ind] # add in diurnal & IGRF (core)
println("Error for scalar Mag 4: ",round(std(map_val-mag_4_c),digits=2)," nT")

### Create the filter model
Next, we do a rough analysis to estimate appropriate filter parameters, and then we initialize a filter model. Specifically, the autocorrelation between the measured (`mag_4_c`) and true (`map_val`) magnetic values is evaluated. This provides information for setting the magnitudes of the measurement variance `meas_var` and First-order Gauss-Markov (FOGM) standard deviation (`fogm_sigma`) and time constant (`fogm_tau`). These are all approximate, and may not be consistently appropriate for the entire flight line.

In [ ]:
(sigma, tau) = get_autocor(mag_4_c-map_val)

In [ ]:
(P0,Qd,R) = create_model(traj.dt,traj.lat[1];
                         init_pos_sigma = 0.1,
                         init_alt_sigma = 1.0,
                         init_vel_sigma = 1.0,
                         meas_var       = 134^2, # increase for bad mag
                         fogm_sigma     = 134,
                         fogm_tau       = 10.0);

### Run the filter
Here, we choose an extended Kalman filter (`:ekf`) and compute:
- `crlb_out`, the Cramér–Rao lower bound error
- `ins_out`, the error when using the INS alone (dead reckoning)
- `filt_out`, the error when using the INS + magnetic measurements (MagNav)

In [ ]:
mag_use = mag_4_c # specify using Mag 4 for navigation
mag_use .+= map_val[1]-mag_use[1] # remove initial DC offset
(crlb_out,ins_out,filt_out) = run_filt(traj,ins,mag_use,itp_mapS,:ekf;P0,Qd,R,core=true); # run the filter

### Plot the results
Despite the 149 m distance-root-mean-square (DRMS) error, the navigation does not go too far awry for the linear model. 

In [ ]:
p3 = plot_map(mapS;legend=false); # plot map background
plot_filt!(p3,traj,ins,filt_out;show_plot=false); # plot GPS, INS (after zeroing), and filter
plot!(p3,legend=:topleft) # move legend as desired

### Corrupting fields 
In the below plot, there are some spiky excursions in the TL-compensated magnetometer that do not occur on the map. Here we can easily compare the two using a MagNav.jl convenience function, `plot_mag_map`.

In [ ]:
p4 = plot_mag_map(traj,mag_use,itp_mapS) # plot magnetometer vs map

However, current sensors for the 💡 strobe lights 💡 picked some of these up. The current sensors have severe high-frequency noise, so we first low-pass filter them using another MagNav.jl convenience functions, `get_bpf` and `bpf_data`.

In [ ]:
lpf     = get_bpf(;pass1=0.0,pass2=0.2,fs=10.0) # get low-pass filter
lpf_sig = -bpf_data(xyz.cur_strb[ind];bpf=lpf)  # apply low-pass filter, sign switched for easier comparison
p5 = plot_basic(xyz.traj.tt[ind],lpf_sig;lab="filtered current for strobe lights") # plot the low-pass filtered strobe light current sensor

## Training a physics-based NN architecture
Now we attempt to improve on the prior results by training a neural network that incorporates the TL model together with an artificial neural network (NN). Specifically, the NN is provided with cabin current sensor information that produces an additive correction to the TL model. Both the NN parameters and the TL coefficients are trainable, which we designate as Model 2c (Model 2b, by comparison, keeps the TL coefficients static).

First, we select all available flight data from flights 1003-1006 into `lines_train`, a subset of which will be held out for testing. It is also possible to evaluate errors on entire flight lines using `lines_test`.

In [ ]:
flts = [:Flt1003,:Flt1004,:Flt1005,:Flt1006] # select flights for training
df_all_3456_train = df_all[(df_all.flight .∈ (flts,) ) .&  # use all flight data
                           (df_all.line.!=1006.08),:]      # except 1006.08
df_nav_3456_test  = df_nav[(df_nav.flight .∈ (flts,) ),:];
lines_train = df_all_3456_train.line; # store training lines
lines_test  = df_nav_3456_test.line;  # store testing  lines

Here, we add current sensors to the usual inputs, making them available to the neural network. 

In [ ]:
features = [:mag_4_uc, :lpf_cur_com_1, :lpf_cur_strb, :lpf_cur_outpwr, :lpf_cur_ac_lo];

The neural network-based compensation parameters, `NNCompParams`, are provided to (and returned by) the training function. These all take default values unless they are overwritten.

In [ ]:
comp_params = NNCompParams(features_setup = features,
                           y_type      = :d,
                           use_mag     = :mag_4_uc,
                           use_vec     = :flux_d,
                           terms       = [:permanent,:induced,:fdm],
                           terms_A     = [:permanent,:induced,:eddy], 
                           sub_diurnal = true,
                           sub_igrf    = true,
                           bpf_mag     = false,
                           norm_type_A = :none,
                           norm_type_x = :standardize,
                           norm_type_y = :standardize,
                           TL_coef     = TL_d_4,
                           model_type  = :m2c,
                           η_adam      = 0.001,
                           epoch_adam  = 300,
                           epoch_lbfgs = 0,
                           hidden      = [8,4]);

In [ ]:
(comp_params,y_train,y_train_hat,err_train,feats) = comp_train(lines_train,df_all,df_flight,df_map,comp_params);

After 300 epochs, Model 2c has 37 nT of error on the training data. We next test the performance on the held-out flight line using the `comp_test` convenience function. Note that there is also a `comp_train_test` convenience function that does both.

In [ ]:
(y_true,y_hat,y_err) = comp_test([1006.08],df_all,df_flight,df_map,comp_params);

We are now in a position to compare the magnetometer readings to the expected map values. The Model 2c results ameliorate the signal excursions that are present in the uncompensated and TL-compensated readings. 

Note that the `detrend` function helps remove any persistent bias in the signal, which does not affect the navigation error.

In [ ]:
tt = (xyz.traj.tt[ind] .- xyz.traj.tt[ind][1]) / 60;
p6 = plot(xlabel="time [min]", ylabel="magnetic field [nT]");
plot!(p6, tt, detrend(map_val  - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Anomaly Map");
plot!(p6, tt, detrend(mag_4_uc - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Uncompensated");
plot!(p6, tt, detrend(mag_4_c  - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Tolles-Lawson");
plot!(p6, tt, detrend(mag_4_uc - y_hat - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Model 2c")

Next, we use the NN output to perform the compensation. Since we selected `y_type = :d` in the NN compensation parameters (`NNCompParams`), we treat the output as the platform component that must be subtracted from the total scalar signal.

In [ ]:
mag_use = mag_4_uc-y_hat # compensate Mag 4 using NN output
mag_use .+= map_val[1]-mag_use[1] # remove initial DC offset
println("TL + NN σ: ",round(std(map_val-mag_use),digits=2))
println("TL σ: ",round(std(mag_4_c.+(map_val[1]-mag_4_c[1])-map_val),digits=2))
(sigma, tau) = get_autocor(mag_use-map_val)

Finally, we use these results in an extended Kalman filter to check navigation performance.

In [ ]:
(P0,Qd,R) = create_model(traj.dt,traj.lat[1];
                         init_pos_sigma = 0.1,
                         init_alt_sigma = 1.0,
                         init_vel_sigma = 1.0,
                         meas_var       = 55^2, # increase for bad mag
                         fogm_sigma     = 55,
                         fogm_tau       = 9);

(crlb_out,ins_out,filt_out) = run_filt(traj,ins,mag_use,itp_mapS,:ekf;P0,Qd,R,core=true);

The physics informed neural network outperforms the linear model!

The notebook continues below to show a similar comparison using a "vanilla" neural network (no linear model). We also provide results for Model 2b, which leaves the TL coefficients as constant in the NN learning process. To reproduce the Model 2b results, simply replace `model_type = :m2c` with `model_type = :m2b` above and replace the `fogm_sigma` and `fogm_tau` values with the corresponding outputs from `plot_autocor`.

Model | Mag Error [nT] | Nav Error [m]
-------- | -------------- | -------
TL | 134  | 149 
TL + NN, Model 2c | 55  | 74 
TL + NN, Model 2b | 39  | 124 
Vanilla NN | 67 | 116 

In [ ]:
p7 = plot_map(mapS;legend=false); # plot map background
plot_filt!(p7,traj,ins,filt_out;show_plot=false); # plot GPS, INS (after zeroing), and filter
plot!(p7,legend=:topleft) # move legend as desired

## Bonus comparison to vanilla neural network
The above approaches used in-cabin vector (Flux D) and scalar (Mag 4) magnetometers, as well as current sensors. Here, we provide the same information to a "vanilla" neural network that does not contain embedded TL information. We have simply added the vector (fluxgate) magnetometer to the feature list and changed the model type to `m1`.

In [ ]:
features = [:TL_A_flux_d, :mag_4_uc, :lpf_cur_com_1, :lpf_cur_strb, :lpf_cur_outpwr, :lpf_cur_ac_lo]
vanilla_comp_params = NNCompParams(features_setup = features,
                                   y_type      = :d,
                                   use_mag     = :mag_4_uc,
                                   use_vec     = :flux_d,
                                   terms       = [:permanent,:induced,:fdm],
                                   terms_A     = [:permanent,:induced,:eddy], 
                                   sub_diurnal = true,
                                   sub_igrf    = true,
                                   bpf_mag     = false,
                                   norm_type_A = :none,
                                   norm_type_x = :standardize,
                                   norm_type_y = :standardize,
                                   TL_coef     = TL_d_4,
                                   model_type  = :m1,
                                   η_adam      = 0.001,
                                   epoch_adam  = 300,
                                   epoch_lbfgs = 0,
                                   hidden      = [8,4]);
(vanilla_comp_params,y_train,y_train_hat,err_train,feats) = comp_train(lines_train,df_all,df_flight,df_map,vanilla_comp_params;silent=true);

In [ ]:
(y_true,y_hat,y_err) = comp_test([1006.08],df_all,df_flight,df_map,vanilla_comp_params);

In [ ]:
mag_use = mag_4_uc-y_hat;
mag_use .+= map_val[1]-mag_use[1];
println("NN σ: ",round(std(map_val-mag_use),digits=2))
println("TL σ: ",round(std(mag_4_c.+(map_val[1]-mag_4_c[1])-map_val),digits=2))
(sigma, tau) = get_autocor(mag_use-map_val)

In [ ]:
tt = (xyz.traj.tt[ind] .- xyz.traj.tt[ind][1]) / 60;
p8 = plot(xlabel="time [min]", ylabel="magnetic field [nT]");
plot!(p8, tt, detrend(map_val  - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Anomaly Map");
plot!(p8, tt, detrend(mag_4_uc - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Uncompensated");
plot!(p8, tt, detrend(mag_4_c  - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Tolles-Lawson");
plot!(p8, tt, detrend(mag_4_uc - y_hat - xyz.igrf[ind] - xyz.diurnal[ind], mean_only=true), label="Vanilla NN")

In [ ]:
(P0,Qd,R) = create_model(traj.dt,traj.lat[1];
                         init_pos_sigma = 0.1,
                         init_alt_sigma = 1.0,
                         init_vel_sigma = 1.0,
                         meas_var       = 67^2, # increase for bad mag
                         fogm_sigma     = 67,
                         fogm_tau       = 11);

(crlb_out,ins_out,filt_out) = run_filt(traj,ins,mag_use,itp_mapS,:ekf;P0,Qd,R,core=true);

The vanilla NN *does* do better than TL alone, but not as well as Model 2c.

We are continuing to work on ways to do more with less data.